*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [69]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-09-07 12:13:50--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   106MB/s    in 0.2s    

2021-09-07 12:13:50 (106 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [30]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [161]:
neigh  = NearestNeighbors(n_neighbors=5,radius=0.8)
columns = {"df_books":df_books.columns,
           "df_ratings":df_ratings.columns,
           "df_Book_Ratings":df_Book_Ratings.columns}
def str_to_int(col,df):
    vocabulary = {}
    for val in df[col].unique():
        add = []
        for a in str(val):
            if str(a) in "0123456789":
                add.append(int(a))
            else:
                add.append(np.sin(ord(a))+np.sin(len(str(val))))
        index = vocabulary.setdefault(val, np.array(add).mean())
    data = df.copy()
    data[col] = data[col].apply(lambda x: vocabulary[x])
    return data[col]


In [162]:
merge = pd.merge(df_books[["title","isbn"]],df_ratings,on=["isbn"])
merge = merge[["user","isbn","title","rating"]]
crush = {col:str_to_int(col,merge) for col in merge.columns if not col in ["rating","user"]}
df_final = pd.DataFrame(data=crush)

In [163]:
df_final["rating"] = merge["rating"]
df_final["user"] = merge["user"]
df_final = df_final[["user","isbn","title","rating"]]

In [133]:
df_final.dtypes

user        int32
isbn      float64
title     float64
rating    float32
dtype: object

In [168]:
y = df_final["rating"]
x = df_final[["title"]]
neigh  = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [139]:
vocabulary = str_to_int("title",df_books)
vocabulary

{'Classical Mythology': 0.41107624015512323,
 'Clara Callan': -0.21323867581287428,
 'Decision in Normandy': 0.8538203487610515,
 'Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It': -0.1470906042564282,
 'The Mummies of Urumchi': 0.1485562940138177,
 "The Kitchen God's Wife": 0.06510504689992631,
 "What If?: The World's Foremost Military Historians Imagine What Might Have Been": -0.2387056098018206,
 'PLEADING GUILTY': 0.8491943362098588,
 'Under the Black Flag: The Romance and the Reality of Life Among the Pirates': -0.12925812036169496,
 "Where You'll Find Me: And Other Stories": 1.2256185500430645,
 'Nights Below Station Street': 1.1737910598066272,
 "Hitler's Secret Bankers: The Myth of Swiss Neutrality During the Holocaust": -0.7086346904188531,
 'The Middle Stories': -0.5335469365387222,
 'Jane Doe': 0.8697604874699687,
 "A Second Chicken Soup for the Woman's Soul (Chicken Soup for the Soul Series)": 1.0875885100662734,
 'The Witc

In [176]:
preds = neigh.kneighbors(np.array([[vocabulary["Where the Heart Is (Oprah's Book Club (Paperback))"]]]))
preds

(array([[0., 0., 0., 0., 0.]]), array([[39579, 39943, 39942, 39638, 39941]]))

In [ ]:
# add your code here - consider creating a new cell for each section of code

In [26]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = df_books.loc[df_books.title==book]
  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [27]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

           isbn  ...        author
706  0446672211  ...  Billie Letts

[1 rows x 3 columns]


KeyError: ignored